In [ ]:
import sys
import os
import cv2
import time
import json
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

# mediapipe for hand tracking
import mediapipe as mp

# gpu or cpu
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print("=" * 30)
print(f"Device in use: {device}")
print("Setup Complete.")
print("=" * 30)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

BASE_DIR = "/content/drive/MyDrive"
DATA_DIR = f"{BASE_DIR}/data"
ASL_DIR = f"{DATA_DIR}/asl_alphabet"

# create directories
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(ASL_DIR, exist_ok=True)

print("✔ Created directories:")
print(DATA_DIR)
print(ASL_DIR)

print("\n============================================================")
print(" HOW TO ATTACH DATASETS TO THIS NOTEBOOK")
print("============================================================")
print("""
STEP 1 — Click 'Add Data' on the right sidebar.

STEP 2 — Search & attach the following datasets:

    ASL Alphabet Dataset (Grassknoted)
    https://www.kaggle.com/datasets/grassknoted/asl-alphabet

STEP 3 — After attaching all datasets, run the next cell to verify paths.
""")

In [ ]:
def find_path(possible_paths):
    for p in possible_paths:
        if os.path.exists(p):
            return p
    return None

# ASL Alphabet dataset
asl_path = find_path([
    # Google Colab paths
    "/content/drive/MyDrive/data/asl_alphabet",
    "/content/drive/MyDrive/data/asl_alphabet_train",
    "/content/asl_alphabet",
])

print("============================================================")
print(" DATASET PATH CHECK RESULTS")
print("============================================================")
print(f"ASL Alphabet found: {asl_path}")

if asl_path: print("✔ ASL Alphabet correctly attached.")
else: print("❌ ASL Alphabet NOT FOUND — attach using 'Add Data'.")

In [ ]:
# extract 3D Landmarks from ASL Images

import os
import json
import numpy as np
import mediapipe as mp
from tqdm import tqdm
import cv2


DATA_DIR = "data/asl_alphabet"  u# pdate to dataset path
OUTPUT_DIR = "output"
MAX_IMAGES_PER_CLASS = 1000

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5
)

# Get class folders (subdirectories in DATA_DIR)
class_folders = sorted([f for f in os.listdir(DATA_DIR) 
                        if os.path.isdir(os.path.join(DATA_DIR, f))])
print(f"Found {len(class_folders)} classes: {class_folders}")

# Create label map
label_map = {folder: idx for idx, folder in enumerate(class_folders)}
print(f"Label map: {label_map}")

# Storage
landmarks_list = []
labels_list = []
no_hand_count = 0

# Process each class
for class_name in tqdm(class_folders, desc="Processing classes"):
    class_path = os.path.join(DATA_DIR, class_name)
    images = [f for f in os.listdir(class_path) 
              if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # Limit to MAX_IMAGES_PER_CLASS
    images = images[:MAX_IMAGES_PER_CLASS]
    
    print(f"Processing class '{class_name}': {len(images)} images")
    
    for img_name in images:
        img_path = os.path.join(class_path, img_name)
        
        # Read and convert image
        img = cv2.imread(img_path)
        if img is None:
            continue
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Process with MediaPipe
        results = hands.process(img_rgb)
        
        if results.multi_hand_landmarks:
            # Extract 21 landmarks (x, y, z for each = 63 values)
            hand_landmarks = results.multi_hand_landmarks[0]
            landmark_values = []
            for landmark in hand_landmarks.landmark:
                landmark_values.extend([landmark.x, landmark.y, landmark.z])
            
            landmarks_list.append(landmark_values)
            labels_list.append(label_map[class_name])
        else:
            no_hand_count += 1
            print(f"  No hand detected: {img_name}")

# Convert to numpy arrays
landmarks_array = np.array(landmarks_list, dtype=np.float32)
labels_array = np.array(labels_list, dtype=np.int32)

# Verify shapes
print(f"\n=== Results ===")
print(f"Landmarks shape: {landmarks_array.shape} (expected: (N, 63))")
print(f"Labels shape: {labels_array.shape} (expected: (N,))")
print(f"Images with no hand detected: {no_hand_count}")

# Save outputs
np.save(os.path.join(OUTPUT_DIR, "landmarks.npy"), landmarks_array)
np.save(os.path.join(OUTPUT_DIR, "labels.npy"), labels_array)

with open(os.path.join(OUTPUT_DIR, "label_map.json"), "w") as f:
    json.dump(label_map, f, indent=2)

print(f"\n✔ Saved to {OUTPUT_DIR}:")
print(f"  - landmarks.npy")
print(f"  - labels.npy")
print(f"  - label_map.json")